<a href="https://colab.research.google.com/github/NatBarbosa/CCG2021_Python-para-geolog-s/blob/main/4_Integracion_de_datos_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. LIBRERÍAS

Las librerías para integrar datos raster y vectoriales incluyen:

1. Las previamente usadas para la manipulación de datos raster

Numpy -- https://numpy.org/

Matplotlib -- https://matplotlib.org/

Gdal -- https://gdal.org/

Rasterio -- https://rasterio.readthedocs.io/en/latest/intro.html#

Glob -- https://docs.python.org/3/library/glob.html

2. Libreías diseñadas para la manipulaciónn de datos vectoriales 

Geopandas - lectura y manipulación de datos vectoriales -- https://geopandas.org/

shapely - https://shapely.readthedocs.io/en/stable/manual.html

Geopandas no está previamente instalada en collab.

In [ ]:
%pip install rasterio
%pip install geopandas

In [ ]:
# Importar todas las librerias
import numpy as np
import rasterio
import os
# Read data from google drive
from google.colab import drive
import copy
import glob
from matplotlib import pyplot as plt
from osgeo import gdal

In [ ]:
#Mount your drive to conect with Colab
drive.mount('/content/drive')

In [ ]:
#Directorio inicial


# 1. LEER EL DEM 

In [ ]:
#Encontrar el DEM localizado en la subcarpeta Dia1/Ejercicio2
os.chdir("/content/drive/MyDrive/PROJECTS/CCG_2021/CourseData/Dia1/Ejercicio2/")
print(os.listdir(cwd))
cwd = os.getcwd()
print("Current working directory: ", cwd)

In [ ]:
#Crear la ruta al DEM
dem_path = os.path.join(cwd, 'AP_20744_FBS_F0060_RT1.dem.tif')

#abrir el archivo con rasterio
dem_open = rasterio.open(dem_path)

In [ ]:
#leerlo como array
dem_array = dem_open.read()
print(f'Forma del areglo original {dem_array.shape}')
#reshape
dem_array = dem_array.reshape(dem_array.shape[1], dem_array.shape[2])
print(f'Forma del areglo modificada {dem_array.shape}')


# 2. LEER LOS DATOS VECTORIALES

Para leer los datos en vector usamos otra librería llamada *Geopandas*

A diferencia de los datos raster, los datos vectoriales estan estructurados como coordenadas geograficas en el espacio. 

* Punto = única coordenada (x,y)
* Línea = distancia entre dos puntos
* Polígono = líneas que unen muchos puntos

In [ ]:
import geopandas as gpd

In [ ]:
#Encontrar los datos
directory = '/content/drive/MyDrive/PROJECTS/CCG_2021/CourseData/Dia2/Ejercicio_shapes/'
paths = glob.glob('/content/drive/MyDrive/PROJECTS/CCG_2021/CourseData/Dia2/Ejercicio_shapes/*.shp')
print(paths)

In [ ]:
#Leer rios

In [ ]:
#Leer cuencas

Geopandas lee la información consignada en un shapefile como una tabla de atributos. Esto se conoce como un 'dataframe'. El dataframe tiene nombres en las columnas que nos sirven para manípular la información contenida de una manera muy similar a una tabla de excel.

Geopandas tiene una función de ploteo integrada que se puede acceder mediante notación punto. 

La visualización de los datos se puede modificar usando atributos del dataframe. 

In [ ]:
#Visualizar los rios usando la columna Strahler como atributo
fig = plt.figure(figsize=(25,6))
ax1 = plt.subplot(131)

plt.title('Rios visualizados por su orden de Strahler')

# visualizar las cuencas
ax2 = plt.subplot(132)

plt.title('Cuencas de rios de Strahler orden 3 ')

# visualizar las cuencas
ax3 = plt.subplot(133)

plt.title('Cuencas de rios de Strahler orden 4 ')

plt.show()


---
Recap

* Para leer datos tipo vector usamos una librería llamada geopandas que lee la tabla de atributos del vector donde esta contenida la información geográfica

* La información geográfica de los puntos que conforman el polígono es usada para visualizar el shapefile

* Se pueden visualizar los diferentes atributos del shapefile cambiando la colummna de visualización en los parámetros de entrada del método de ploteo

---

# 3. COMBINAR DATOS FORMATO RASTER CON DATOS VECTORIALES

Esto no es una operación directa porque la manera de trabajar datos raster y vectores es muy diferentes. Los raster se almacenan como matrices geográficamente localizadas por un borde (xmin, xmax, ymin, ymax). Los vectores, en cambio, son almacenados como geometrías que contienen multiples coordenadas. 

## 3.1. Información geográfica del raster

Para visualizar los dos tipos de datos en un mismo plot tenemos que extraer la geotransformada. La geotransformada es la definición espacial de un raster. Corresponde a un vector de 6 coefficientes. 

* GT(0) x-coordinate of the upper-left corner of the upper-left pixel.
* GT(1) w-e pixel resolution / pixel width.
* GT(2) row rotation (typically zero).
* GT(3) y-coordinate of the upper-left corner of the upper-left pixel.
* GT(4) column rotation (typically zero).
* GT(5) n-s pixel resolution / pixel height (negative value for a north-up image).

In [ ]:
#Extraer coordenadas del DEM - geotransform



[466894.1875, 12.5, 0.0, 467255.25, 0.0, -12.5]


$$ Extent = (xmin, xmax, ymin, ymin)$$

In [ ]:
#Necesitamos el xmin, xmax = xmin + ncols*pixels size, ymin, ymax= ymin+ nrows*pixels size

(466894.1875, 549131.6875, 395605.25, 467255.25)


## 3.2. Plotear juntos usando Matplotlib

Existen muchas librerías diseñadas en plotear vector y raster juntos. En este curso intentamos usar las librerías base para tener mayor libertad de procesamiento y ajustes a nuestras necesidades. 

Para plotear el raster usando su información geográfica definimos el extent


In [ ]:
fig = plt.figure(figsize = (12, 8))
ax1 = plt.subplot(111)



plt.colorbar(img)
plt.show()

---

Recap

* Para combinar información en formato raster y vectorial necesitamos entender con claridad los parametros cartográficos y geográficos de nuetros datos. 

* La información geográfica del raster esta consignada en la geotransformada

* La información geográfica del vector esta consignada como coordenadas X,Y en una tabla de atributos 


---

# 4. EXTRAER INFORMACIÓN DEL RASTER USANDO UN ÁREA DEFINIDA POR UN VECTOR 

Para extraer la información por cuenca, el procedimiento es el siguiente:

1. Extraer la geometría de cada una de las cuencas
2. Extraer los valores del raster dentro de la geometría del polígono de la cuenca maskeando los valores del raster que no corresponden con el polígono



In [ ]:
#Geometria de todas las cuencas


In [ ]:
#Geometria de la primera cuenca


**shapely** es una librería diseñada para el procesamiento de datos vector y nos permite mapear el polígono en el raster. Shapely usa un formato de intercambio de datos llamado JSON (JavaScript Object Notation). JSON es un formato de texto completamente independiente de lenguaje y es muy usado en el procesamiento de información geográfica tipo vector.

In [ ]:
from shapely.geometry import mapping
from rasterio.mask import mask

In [ ]:
#Transformar las coordenadas a JSON



In [ ]:
#Crear una mascara para los valores del DEM que no pertenecen al poligono


In [ ]:
#Plotear los valores de elevacion en la cuenca 1
plt.imshow(cuenca1.reshape(cuenca1.shape[1], cuenca1.shape[2]))
plt.colorbar()

In [ ]:
#Extraer alturas del raster enmascarado! 


Las elevaciones de la cuenca son: [ 318  318  318 ... 1405 1428 1446]


In [ ]:
#Cual es esta cuenca?
fig = plt.figure(figsize = (12, 8))
ax1 = plt.subplot(111)
img = plt.imshow(dem_array, extent=extent, origin='upper', cmap = 'gist_earth')
basins_strahler3.plot(ax=ax1, column='basin_id', edgecolor='white', cmap = 'jet', legend = True)
#plt.colorbar(img)
plt.show()

---
Recap

* Para extraer la información del raster contenida en una geometría determinada hacemos una máscara de los valores que  se intersectan con la geometría de interés.

---

#5.  EJEMPLO - CÁLCULO DE VARIABLES MORFOMÉTRICAS DE LA CUENCA 

De manera rápida, vamos a caracterizar esta cuenca usando las elevaciones del DEM. 









In [ ]:
#Calcular informacion estadistica sobre la cuenca 
h_min = 
h_max = 
basin_area = 
h_promedio = 


In [ ]:
print(f'elevacion minima de la cuenca: {h_min}')
print(f'elevacion minima de la cuenca: {h_max}')
print(f'Area de la cuenca: {basin_area/1000000} en km2')
print(f'Elevacion promedio de la cuenca: {h_promedio} m.s.n.m')

elevacion minima de la cuenca: 316
elevacion minima de la cuenca: 1492
Area de la cuenca: 97.36140625 en km2
Elevacion promedio de la cuenca: 780.1337092951038 m.s.n.m


## 5.1. Frecuencias altimétricas

Las frecuencias altimétricas se usan para describir, en altitudes sucesivas, las frecuencias de ciertos niveles de elevaciones.



In [ ]:
#1. Crear un array igualmente espaciado de alturas entre la minima y la maxima en la cuenca
h_range = 

In [ ]:
#Plotear frecuencias altimetricas
fig= plt.figure()
ax1 = plt.subplot(111)
ax1.hist(cuenca1_elevacion, bins = h_range )
ax1.set_xlabel('Area: a/A')
ax1.set_ylabel('Elevacion: h/H')


## 5.2. Curva Hipsométrica

La curva hipsométrica (Strahler 1952) permite conocer la distribucién de masa en la cuenca desde arriba hacia abajo. Se obtiene calculando:
* x como las diferentes alturas alturas de la cuenca referidos a la máxima altura de la misma
* y como los valores de area que se encuentran por encima de las alturas correspondientes, referidos al área total de cuenca. 

De esta forma se utilizan valorse relativos (porcentuales) lo que hace posible comparar curvas de diferentes cuencas.

<img src="https://drive.google.com/uc?export=view&id=1okz5ELENyC8cieaAHupZvgBfhN7AJOO2"  width=200 />




In [ ]:
#2. Calcular el area entre cada intervalo
#Lista vacia para llenar con el area por encima de una altura definida


In [ ]:
#Normalizar las elevaciones a la maxima elevacion


In [ ]:
#Curva hpsometrica
fig= plt.figure(figsize=(15,6))
ax1 = plt.subplot(121)

ax1.plot(           )


ax1.set_xlabel('Area: a/A')
ax1.set_ylabel('Elevacion: h/H')

#Plot altitudes
ax2 = ax1.twinx()
ax2.plot(x, h_range)
ax2.set_ylabel('Altura en m.s.n.m')

ax3 = plt.subplot(122)
ax3.hist(cuenca1_elevacion, bins = h_range, orientation='horizontal', alpha=0.2)

plt.show()

## 5.3. Elevación media

La elevación media se determina con base en la curva hipsométrica. La elevación media de la cuenca es el equivalente al 50% del area de la cuenca. La altura media es la elevación promedio referida al nivel de la estación de aforo de la boca de cuenca. 

In [ ]:
#Identificar el valor aproximado del 50% en x


In [ ]:
#Obtener el valor de elevacion al punto x con aproximado al 50% del area de la cuenca
elevacion_media = 
print(f'La elevacion media de la cuenca es {elevacion_media}')

## 5.4. Indices Hipsométrico

El índice hipsométrico se calcula como.

$$HI = \frac{h_{media} - h_{min} }{ h_{max} - h_{min}} $$




In [ ]:
# Indice hipsometrico


---
Recap

* Hemos calculado variables morfométricas comumemente usadas para describir la morfologia de una cuenca en relación con la elevación a partir de las elevaciones del DEM y un shape file del perímetro de una cuenca. 

* El objetivo de caracterizar una cuenca desde el punto de vista geomorfológico es identicar patrones y anomalías que puedan ser relacionadas y/o interpretadas como influencia de la geología, el clima y/o el movimiento del agua. 

* Ejemplo. Distribución del índice hipsométrico en el valle del Zerafshan, Tian Shan. 

<img src="https://drive.google.com/uc?export=view&id=1eP9cBWrHiUkalBGXCBxmtlMEO1AMNhd3"  width=1100 />

---

--- 
# RESUMEN DÍA 2

* Usamos lo aprendido el día 1 sobre la manipulación de datos raster para leer, visualizar y manipular imagenes satelitales. 

* Usamos lo aprendido sobre la información geográfica consignada en la metadata de los datos para plotear en un mismo gráfico datos en formato vector y raster.

* Usamos la geometría de un dato en formato vector para extraer información puntual del DEM.

* Usamos la información extraida del DEM para hacer algunos cálculos matemáticos y gráficos. 

---